##Warning! Almost nothing is in functions

In [1]:
#provide directory in which BASS is located
bass_folder_loc = '/Users/morganfine-morris/Documents/Repos/bass'
#uncomment '%matplotlib inline' below see plots inline (and remove blocking). 
#If you change your mind after running it, you must restart kernel
#%matplotlib inline 
#%matplotlib qt

import sys
import os
import os.path as path
import pandas as pd
import matplotlib.pyplot as plt
# append the path to the folder containing bass 
# (which should prob. contain a __init__.py file.) to pythonpath
sys.path.append(bass_folder_loc) 
import bass
reload(bass)

BASS ready!
BASS ready!


<module 'bass' from '/Users/morganfine-morris/Documents/Repos/bass/bass.pyc'>

In [2]:
#convert the sel100.csv into something usable (although i manually removed the units.)
ecgcsv = "./Physionet_ECGs/sel100.csv"
sel100 = pd.DataFrame.from_csv(ecgcsv, header=0)
new = pd.DataFrame(sel100.values, sel100.index.values)
new.to_csv("./Physionet_ECGs/sel_100_tabsep.csv", sep='\t',header=False)

In [3]:
# is long QT episodic?
# 12 variants
# automated QT detector. Are they at risk? Which variant do they have?

In [3]:
#initialize new file
Data = {}
Settings = {}
Results ={}

############################################################################################
#manual Setting block
Settings['folder']= r"./Physionet_ECGs"
Settings['Label'] = r'sel_100_tabsep.csv'
Settings['Output Folder'] = 'output'

#transformation Settings
Settings['Absolute Value'] = True #Must be True if Savitzky-Golay is being used

Settings['Bandpass Highcut'] = 30 #in Hz
Settings['Bandpass Lowcut'] = 100 #in Hz
Settings['Bandpass Polynomial'] = 4 #integer

Settings['Linear Fit'] = False #between 0 and 1 on the whole time series
Settings['Linear Fit-Rolling R'] = 0.75 #between 0 and 1
Settings['Linear Fit-Rolling Window'] = 1000 #window for rolling mean for fit, unit is index not time
Settings['Relative Baseline'] = 0 #default 0, unless data is normalized, then 1.0. Can be any float

Settings['Savitzky-Golay Polynomial'] = 4 #integer
Settings['Savitzky-Golay Window Size'] = 251 #must be odd. units are index not time

#Baseline Settings
Settings['Baseline Type'] = r'static' #'linear', 'rolling', or 'static'
#For Linear
Settings['Baseline Start'] = 50.04 #start time in seconds
Settings['Baseline Stop'] = 50.18 #end time in seconds
#For Rolling
Settings['Rolling Baseline Window'] = 1 # in seconds. leave as 'none' if linear or static

#Peaks
Settings['Delta'] = 0.25
Settings['Peak Minimum'] = -1 #amplitude value
Settings['Peak Maximum'] = 1 #amplitude value

#Bursts
Settings['Burst Area'] = False #calculate burst area
Settings['Exclude Edges'] = False #False to keep edges, True to discard them
Settings['Inter-event interval minimum (seconds)'] = 0.0100 #only for bursts, not for peaks
Settings['Maximum Burst Duration (s)'] = 10 
Settings['Minimum Burst Duration (s)'] = 0
Settings['Minimum Peak Number'] = 1 #minimum number of peaks/burst, integer
Settings['Threshold']= 1.0 #linear: proportion of baseline. 
                           #static: literal value.
                           #rolling, linear ammount grater than rolling baseline at each time point.

#Outputs
Settings['Generate Graphs'] = False #create and save the fancy graph outputs


#Settings that you should not change unless you are a super advanced user:
#These are settings that are still in development
Settings['Graph LCpro events'] = False
Settings['File Type'] = r'Plain' #'LCPro', 'ImageJ', 'SIMA', 'Plain', 'Morgan'
Settings['Milliseconds'] = False
############################################################################################

In [16]:
#find data file
data_folder = bass_folder_loc
dir_content = os.listdir(data_folder)
list(enumerate(dir_content))

[(0, '.git'),
 (1, '.ipynb_checkpoints'),
 (2, '__init__.py'),
 (3, '__init__.pyc'),
 (4, 'Bass - Flowchart.jpg'),
 (5, 'Bass - Flowchart.odp'),
 (6, 'Bass - Flowchart.pdf'),
 (7, 'bass.py'),
 (8, 'bass.pyc'),
 (9, 'Change Log'),
 (10, 'Kitchen Sink-Bass.ipynb'),
 (11, 'licence.txt'),
 (12, 'pyeeg.py'),
 (13, 'pyeeg.pyc'),
 (14, 'rat34_ECG.txt'),
 (15, 'rat34_Settings.csv'),
 (16, 'README.md'),
 (17, 'Single Wave- Basic.ipynb'),
 (18, 'Single Wave- Interactive.ipynb'),
 (19, 'Single Wave-Interactive Protocol.txt')]

In [17]:
data_file = dir_content[14]
settings_file = dir_content[15]

In [44]:
Data = {}
Results = {}
Settings = {}

#process Settings file (bass load_settings func file didn't seem to work.)
settings_file_path = path.join(data_folder,settings_file)
for i in open(settings_file_path):
    stripped =  i.strip()
    elements = stripped.split("\n")
    for elem in elements:
        if 'Settings' in elem:
            continue
        name, val = elem.split('\t')
        try:
            val = int(val)
        except:
            try:
                val=float(val)
            except:
                pass
            
        Settings[name] = val
        
Settings['plots folder'] = 'output'
Settings['Output Folder'] = 'output'
Settings['folder'] = data_folder
Settings["Minimum Peak Number"] = 1
Settings['Threshold'] = 0.03#0.04 #this catches most of the T peaks
#these just make it worse
#Settings['Savitzky-Golay Window Size']=251
#Settings['Savitzky-Golay Polynomial'] = 6
pd.Series(Settings) #view settings

Absolute Value                                                                     True
Bandpass Highcut                                                                   none
Bandpass Lowcut                                                                    none
Bandpass Polynomial                                                                none
Baseline Type                                                                    static
Burst Area                                                                        False
Delta                                                                              0.25
Exclude Edges                                                                      True
File Type                                                                         Plain
Generate Graphs                                                                   False
Graph LCpro events                                                                False
Inter-event interval minimum (se

In [4]:
reload(bass)
Data, Settings = bass.load_wrapper({},Settings)
#Data, Settings = bass.transform_wrapper(Data, Settings) #b/c I can't find smoothing settings that don't make it worse
Data['trans'] = Data['original']
Data, Settings, Results = bass.baseline_wrapper(Data, Settings, Results)

BASS ready!
Made plots folder
Data Loaded
Sampling Rate= 0.004 (sec/frame)
sel_100_tabsep.csv is 899.996 seconds long.


In [12]:
#event detection
Results = bass.event_peakdet_wrapper(Data, Settings, Results)
Results = bass.event_burstdet_wrapper(Data, Settings, Results)

#clip the data from s seconds to e seconds
s=1
e=20
Data_clipped = {'original':Data['original'][s:e], 'trans':Data['trans'][s:e]}
# graph_ts calls show, so unless you ran %matplotlib inline, 
# notebook will block until you close the plot window
bass.graph_ts(Data_clipped, Settings, Results, roi="Mean2") 

Mean1has no bursts.
Mean2has no bursts.
No Bursts Found in any time series.


In [6]:
Data['original'].keys()


Index([u'Mean1', u'Mean2'], dtype='object')

#Getting Q, R, S, and T in the DataFrame and Ordered according to Time

In [164]:
peaks_df = Results['Peaks']['Mean1'] # R wave location
valleys_df = Results['Valleys']['Mean1'] # Q and S wave locations
burst_ends = Results['Bursts']['Mean1']['Burst End'] #burst ends indicate approx. T wave location

# make new DataFrame for Burst Ends which is indexed by Burst Ends 
# b.c. Results['Bursts']['Mean1'] is indexed by Burst Start.
bursts_df = pd.DataFrame(burst_ends.values, index=burst_ends.values, columns=['Burst Ends'])

#combine peaks and valleys dfs and sort by time
# this will allow us to easily find peaks, 
# and find valleys before and after.
peaks_df['time'] = peaks_df.index
valleys_df['time'] = valleys_df.index
bursts_df['time'] = bursts_df.index
peaks_df['amplitude'] = peaks_df["Peaks Amplitude"]
valleys_df['amplitude'] = valleys_df["Valley Amplitude"]
bursts_df['amplitude'] = float(Settings['Threshold'])

#Label R and T. Unfortunatly, setting Q and S (the valleys) is much harder (See next cell).
peaks_df['letter'] = 'R'
bursts_df['letter'] = "T"

combined = peaks_df.append(valleys_df).sort('time')

#now combine w. Burst Ends and remove unnecessary columns, sort by time again
times = combined.append(bursts_df).drop([u'Burst Ends', u'Intervals', u'Peaks Amplitude', u'Valley Amplitude'],axis=1)
sorted_times = times.sort('time')
#remove na vals only from amplitude col.
sorted_times['amplitude'] = sorted_times['amplitude']
sorted_times.head()

,amplitude,letter,time
Time,,,
0.12950,-0.487,NaN,0.12950
0.13475,1.410,R,0.13475
0.13825,-0.883,NaN,0.13825
0.15575,0.030,T,0.15575
0.30125,-0.455,NaN,0.30125


The following is a hack-y way to set the Q and S valleys, but it works. Later I might try to do it with booleans. <b>Feel free to try another way to do this!!! (Actually, really, please do. This method is pretty slow.)</b>

In [165]:
#reindex by integer
sorted_times['num'] = range(len(sorted_times))
sorted_times_reindexed = sorted_times.set_index('num')

letters = pd.DataFrame(index=sorted_times_reindexed.index, columns=['letters'], dtype=str)
for i in sorted_times_reindexed.index:
    
    #try to get the previous and next values with error handeling for edge cases
    try:
        prev_value = sorted_times_reindexed.loc[i-1, "letter"]
    except:
        prev_value=''
    try:
        next_value = sorted_times_reindexed.loc[i+1,"letter"]
    except:
        next_value=""
    
    #letters['time'][i] = sorted_times_reindexed.loc[i,'time']
    if (next_value == "T") and (prev_value == "R"):
        letters['letters'][i] = "S"
    elif next_value == "R":
        letters['letters'][i] = "Q"
    else:
        letters['letters'][i] = sorted_times_reindexed.loc[i,'letter']
        
sorted_times_reindexed['letter'] = letters['letters']
sorted_times_reindexed.head(13) #see that valleys which are not immediately before or after R are left as NaN!

,amplitude,letter,time
num,,,
0,-0.487,Q,0.12950
1,1.410,R,0.13475
2,-0.883,S,0.13825
3,0.030,T,0.15575
4,-0.455,Q,0.30125
5,1.514,R,0.30700
6,-0.934,S,0.31050
7,0.030,T,0.32575
8,-0.376,Q,0.47350


In [147]:
sorted_times_time_indexed = sorted_times_reindexed.set_index('time')
final = sorted_times_time_indexed.dropna()
final.head()

,amplitude,letter
time,,
0.12950,-0.487,Q
0.13475,1.410,R
0.13825,-0.883,S
0.15575,0.030,T
0.30125,-0.455,Q


In [153]:
s1=0
e1=20
final[s1:e1].plot()
plt.show()